*Copyright (c) Microsoft Corporation. All rights reserved.*

*Licensed under the MIT License.*

# Sentiment Analysis using Classical Explainer 

In [ ]:
import pandas as pd
import nlp

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

from interpret_text.experimental.classical import ClassicalTextExplainer

pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option("max_rows", None)

In [ ]:
train = nlp.load_dataset("emo", split = "train")

In [ ]:
id2label = {0: 'others', 1: 'happy', 2: 'sad', 3: 'angry'}
labels=list(id2label.values())
label2id = {}
for i,label in enumerate(labels):
    label2id[label]=i

In [ ]:
data={'text':[],
     'emotion':[]}
for val in train:
    if id2label[val['label']]!='others':
        data['text'].append(val['text'])
        data['emotion'].append(id2label[val['label']])
        
data = pd.DataFrame(data)

In [ ]:
X_str = data['text']
ylabels = data['emotion']

In [ ]:
# Create explainer object that contains default glassbox classifier and explanation methods
explainer = ClassicalTextExplainer()
label_encoder = LabelEncoder()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_str, ylabels, train_size=0.8, test_size=0.2)
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
print("X_train shape =" + str(X_train.shape))
print("y_train shape =" + str(y_train.shape))
print("X_train data structure = " + str(type(X_train)))

In [ ]:
classifier, best_params = explainer.fit(X_train, y_train)

In [ ]:
# obtain best classifier and hyper params
print("best classifier: " + str(best_params))

In [ ]:
mean_accuracy = classifier.score(X_test, y_test, sample_weight=None)
print("accuracy = " + str(mean_accuracy * 100) + "%")
y_pred = classifier.predict(X_test)
[precision, recall, fscore, support] = precision_recall_fscore_support(y_test, y_pred,average='macro')

In [ ]:
# Enter any document or a document and label pair that needs to be interpreted
document = "There is no limit to what we, as women can acomplish"

In [ ]:
# Obtain the top feature ids for the selected class label
explainer.preprocessor.labelEncoder = label_encoder

In [ ]:
local_explanation = explainer.explain_local(document)

In [ ]:
y = classifier.predict(document)
predicted_label = label_encoder.inverse_transform(y)
local_explanation = explainer.explain_local(document, predicted_label)

In [ ]:
from interpret_text.experimental.widget import ExplanationDashboard

In [ ]:
ExplanationDashboard(local_explanation)